# Geocode Exploration

* Goal: Can I specify a location and then get the longitude and latitude I need?

### PoC: Addresses mom and dad have lived at
1. 785 Ivy Glendale
2. Ridgeway Avenue
3. Peppermil Lane
4. Caracas at La Vista
5. Trailbridge Drive

In [96]:
#libraries
import pandas as pd
import geopy
from geopy import geocoders
from geopy.geocoders import Nominatim
import folium
from folium import plugins



In [4]:
#Initialize with username
gn = geocoders.GeoNames(username = 'BenjaminBoughton')

geolocator = Nominatim(user_agent = 'Nav_app')

In [144]:
current_location = geolocator.geocode('calvert cochabamba bolivia')
current_location



Location(UE Cochabamba Cooperative School - Calvert, Calle Vaticano, Mesadilla, Distrito 1, Tunari, Cochabamba, Kanata, Cercado, Cochabamba, 538, Bolivia, (-17.370959300000003, -66.13233570762681, 0.0))

In [145]:
# First there will be a prompt for locations - so these will be known ahead of time before constructing df
locations = ['calvert cochabamba bolivia']


locations_df = pd.DataFrame(columns = ['Region', 'Name', 'Longitude', 'Latitude'], 
                            index = [i for i in range(0, len(locations))])



#Populate df with geocode
for i in range(0, len(locations_df)):
    current_location = geolocator.geocode(locations[i])
    
    
    region = current_location.address.split(',')[-1].lstrip()
    name = current_location.address
    longitude = current_location.longitude
    latitude = current_location.latitude
    
    
    
    locations_df.loc[i, 'Region'] = region
    locations_df.loc[i, 'Name'] = name
    locations_df.loc[i, 'Longitude'] = longitude
    locations_df.loc[i, 'Latitude'] = latitude

    

In [146]:
#Now we have our df - let's build the map we need!

locations_df.reset_index(drop=False, inplace=True)
locations_df.rename(columns={'index': 'icon_num'}, inplace=True)
locations_df['icon_num'] = locations_df['icon_num'] + 1
long_mean = locations_df['Longitude'].mean()
lat_mean = locations_df['Latitude'].mean()

#Need list of lists of long lats
longitudes = [long for long in locations_df['Longitude']]
latitudes = [lat for lat in locations_df['Latitude']]

long_lats = [[latitudes[i], longitudes[i]] for i in range(0, len(longitudes))]


In [147]:
#Initial map creation
map_plot_route = folium.Map(location = [lat_mean, long_mean], zoom_start = 14, tiles='CartoDB positron')

#Markers
locations_df.apply(lambda row: folium.Marker(location = [row['Latitude'], row['Longitude']],
                                         popup=row['Name'],
                                         icon=plugins.BeautifyIcon(number = row['icon_num'],
                                                                  border_color='transparent',
                                                                  border_width =0.5,
                                                                  text_color='#A12830',
                                                                  inner_icon_style = 'font-size:20px;margin:0px;')).add_to(map_plot_route), axis=1)


#Route
folium.PolyLine(long_lats).add_to(map_plot_route)

#Save as html
map_plot_route.save('map.html')




In [148]:



map_plot_route

